In [56]:
from openai import OpenAI
import pandas as pd
import os
import json


df = pd.read_excel('/Users/andreasbrogaard/Documents/dsp_project/EDC w. description.xlsx')

zeros = [0] * 14
GPT_response = pd.DataFrame([zeros], columns= ['location', 'Number of bathrooms', 'Condition of the house', 'view', 'amenities.Balcony',  'amenities.Garden',  'amenities.Parking',  'amenities.Swimming pool',  'amenities.Fireplace',  'amenities.Garage',  'amenities.Basement',  'amenities.Elevator',  'amenities.Air conditioning',  'amenities.Heating Type'])

print(GPT_response)

   location  Number of bathrooms  Condition of the house  view  \
0         0                    0                       0     0   

   amenities.Balcony  amenities.Garden  amenities.Parking  \
0                  0                 0                  0   

   amenities.Swimming pool  amenities.Fireplace  amenities.Garage  \
0                        0                    0                 0   

   amenities.Basement  amenities.Elevator  amenities.Air conditioning  \
0                   0                   0                           0   

   amenities.Heating Type  
0                       0  


In [2]:
system_message = ("You are a parameter identifier you are to identify the parameters. Your output is to json and only json. do not nest any variables in the json, do not make an introduction or summary of your response")

parameters_schema = {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": 'Divide locations into categories such as neighborhoods, districts, or proximity to amenities (downtown, suburban, rural).',
        },
        "Number of bathrooms": {
            "type": "string",
            "enum": ['1-1.5 bathrooms', '2-2.5 bathrooms', '3+ bathrooms'],
            "description": 'The stated number of bathrooms. If no number of bathrooms is provided, return "not_stated".',
        },
        "Condition of the house": {
            "type": "string",
            "enum": ["new construction", "renovated", "fixer-upper"],
            "description": 'Return the condition of the house as mentioned".',
        },
        "amenities": {
            "type": "array",
            "Balcony": {
                "type": "Boolean",
            },
            "Garden": {
                "type": "Boolean",
            },
            "Parking": {
                "type": "Boolean",
            },
            "Swimming pool": {
                "type": "Boolean",
            },
            "Fireplace": {
                "type": "Boolean",
            },
            "Garage": {
                "type": "Boolean",
            },
            "Basement": {
                "type": "Boolean",
            },
            "Elevator": {
                "type": "Boolean",
            },
            "Air conditioning": {
                "type": "Boolean",
            },
            "Heating Type": {
                "type": "string",
            },
            "description": 'Return the amenities mentioned in the description.',
        },
        "view": {
            "type": "string",
            "enum": ["city", "sea", "mountain", "no view", "not_stated"],
            "description": 'Return the view mentioned in the description.',
        },
    },
    #"required": ["location", "budget_level", "purpose"],
}




In [7]:
OPENAI_API_KEY = open('/Users/andreasbrogaard/Documents/dsp_project/.env','r').read()

client = OpenAI(
    api_key = OPENAI_API_KEY
)
for i in range(0, 10):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message + "use the following schema to create a response" + str(parameters_schema)},
            {"role": "user", "content": df['Description'].iloc[i]}
        ],
        temperature= 0.5
    )
    
    response = completion.to_dict()['choices'][0]['message']['content']

    json_response = json.loads(response)

    GPT_response = GPT_response.append({completion.choices[0].message['content']}, ignore_index=True)


ChatCompletion(id='chatcmpl-9Hb8YcFhJ2KqYHtz6oGQgXbJjvXjH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "location": "Ø-gadekvarteret",\n    "Number of bathrooms": "not_stated",\n    "Condition of the house": "well maintained",\n    "amenities": {\n        "Balcony": false,\n        "Garden": false,\n        "Parking": false,\n        "Swimming pool": false,\n        "Fireplace": false,\n        "Garage": false,\n        "Basement": true,\n        "Elevator": false,\n        "Air conditioning": false,\n        "Heating Type": "not_stated"\n    },\n    "view": "no view"\n}', role='assistant', function_call=None, tool_calls=None))], created=1713981358, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=129, prompt_tokens=891, total_tokens=1020))


In [48]:
import json

response = completion.to_dict()['choices'][0]['message']['content']

json_response = json.loads(response)

df_json_response = pd.json_normalize(json_response)


print(df_json_response)

GPT_response = GPT_response.append(df_json_response, ignore_index=True)



          location Number of bathrooms Condition of the house     view  \
0  Ø-gadekvarteret          not_stated        well maintained  no view   

   amenities.Balcony  amenities.Garden  amenities.Parking  \
0              False             False              False   

   amenities.Swimming pool  amenities.Fireplace  amenities.Garage  \
0                    False                False             False   

   amenities.Basement  amenities.Elevator  amenities.Air conditioning  \
0                True               False                       False   

  amenities.Heating Type  
0             not_stated  


AttributeError: 'DataFrame' object has no attribute 'append'

In [60]:
response = completion.to_dict()['choices'][0]['message']['content']

json_response = json.loads(response)

GPT_response = GPT_response.append(json_response, ignore_index=True)

print(GPT_response)


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
GPT_response.to_pickle('House_data_w_GPT.pkl')
